In [34]:
!pip3 install sklearn
!pip3 install matplotlib


In [60]:
import pandas as pd
import numpy as np

import os
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import string

In [61]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fabricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
print(stopwords.words('portuguese'))

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

In [63]:
dataTRUE = "Data/size_normalized_texts/true/"

list = os.listdir(dataTRUE)
number_files_true = len(list)

In [64]:
dataFAKE = "Data/size_normalized_texts/fake/"

list = os.listdir(dataTRUE)
number_files_fake = len(list)

In [65]:
TEXT = []
LABEL = []

In [66]:
# Load data true
for i in range(number_files_true):
    title = list[i]
    LABEL.append(1)
    with open(dataTRUE + title,'r') as reader:

        doc = reader.read()
        doc.lower()
        doc.split()
        reader.close
        TEXT.append(doc)

In [67]:
# Load data fake
for i in range(number_files_fake):
    title = list[i]
    LABEL.append(0)
    with open(dataFAKE + title,'r') as reader:

        doc = reader.read()
        doc.lower()
        doc.split()
        reader.close
        TEXT.append(doc)

In [68]:
print(f"True {number_files_true}")
print(f"Fake {number_files_fake}")

True 3600
Fake 3600


In [69]:
assert len(TEXT) == len(LABEL)

In [70]:
print(f"Label {len(LABEL)} - TEXT {len(TEXT)}")

Label 7200 - TEXT 7200


In [71]:
def vectors_and_df(text, label):
    """
    creates vectors for the text files and returns dataframe with articles as word vectors 
    """
    
    cv = TfidfVectorizer()
    cv.fit(text)
    corpus_vecs = cv.transform(text)
    
    return pd.DataFrame(corpus_vecs.todense(), index=label, 
                        columns=cv.get_feature_names()), cv

In [72]:
# Store results into dataframe, keep cv for later prediction
df, cv = vectors_and_df(TEXT, LABEL)

/home/fabricio/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [73]:
X = df
y = df.index

In [74]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.30,shuffle=True, random_state=42)

In [75]:
model = LogisticRegression(max_iter=300)
model.fit(Xtrain, ytrain)

LogisticRegression(max_iter=300)

In [76]:

pred = model.predict(Xtest)
pred

array([1, 1, 0, ..., 0, 0, 1])

In [77]:
# Create the predictions for Y training data
preds = model.predict(Xtest)

print(model.score(Xtrain, ytrain))
print(f"Acuracia {model.score(Xtest, ytest)}")

0.964484126984127
Acuracia 0.8949074074074074


In [78]:
print("Accuracy:",round(metrics.accuracy_score(ytest, preds)*100, 2),'%')
print("Precision:",round(metrics.precision_score(ytest, preds)*100, 2), '%')

Accuracy: 89.49 %
Precision: 90.72 %


In [79]:
matrix = confusion_matrix(ytest, preds)
matrix

array([[975,  98],
       [129, 958]])

In [80]:
def predict(new_text):
    
    """
    Takes the pre-trained model pipeline and predicts new artist based on unseen text.
    
    Parameters
    ----------
    model : Trained scikit-learn model pipeline.
    new_text : str
    
    Returns
    ---------
    prediction : str
    
    """
    article = [new_text]
    # transform song into vector matrix
    new_article_vecs = cv.transform(article)
    ynew = new_article_vecs.todense()
    print(ynew.shape)
    prediction = model.predict(ynew)
    
    return prediction[0]

In [81]:
TEXT = """A Câmara aprovou nesta quarta-feira, 4, o projeto de lei que cria um piso salarial para profissionais da enfermagem, a nível nacional, que varia de R$ 2.375 a R$ 4.750. O piso deve gerar um custo de até R$ 18 bilhões aos serviços de saúde públicos e privados de acordo com o presidente da Casa, deputado Arthur Lira (PP-AL). O texto foi aprovado por 449 votos favoráveis a 12, apesar de o governo Bolsonaro ter se posicionado contra.De origem do Senado, o PL determina que o piso da categoria deverá ser reajustado anualmente com base no Índice Nacional de Preços ao Consumidor (INPC). Além disso, fica assegurada a manutenção de salários eventualmente superiores ao valor inicial sugerido, independentemente da jornada de trabalho para a qual o profissional tenha sido contratado.Contudo, de acordo com a relatora da proposta, deputada Carmen Zanotto (Cidadania-SC), o projeto só irá à sanção presidencial após promulgação da Proposta de Emenda à Constituição (PEC) 122/15, aprovada em fevereiro deste ano, que proíbe a União de criar despesas aos demais entes federativos sem prever a transferência de recursos para o custeio.Desde segunda-feira, 2, profissionais da enfermagem circulavam pela Câmara e conversavam com parlamentares, numa mobilização que levou a um amplo apoio à proposta entre deputados. No plenário, parlamentares favoráveis ao projeto argumentaram que os profissionais de saúde merecem reconhecimento por terem estado na linha de frente do combate à pandemia de covid-19.O deputado Tiago Mitraud (MG), líder do Novo, único partido contrário ao piso, afirmou que a proposta é eleitoreira e tem alto impacto orçamentário. "Este projeto vai acabar com a saúde brasileira porque vamos ver as Santas Casas fechando, leitos de saúde fechando e os profissionais que hoje estão aqui lutando pelo piso desempregados porque os municípios não conseguirão pagar esse piso", criticou.Na avaliação do líder do governo na Câmara, deputado Ricardo Barros (PP-PR), a criação do piso representa um grande desafio para os cofres públicos. Barros disse que, como não havia indicação da origem dos recursos para custear a proposta, a orientação do governo seria contrária. "Neste momento, o projeto não atende à Lei de Responsabilidade Fiscal (LRF), e por isso o voto do governo é contrário", declarou.No entanto, como a aprovação do projeto era inevitável, Barros garantiu que o governo se manterá empenhado em buscar fontes de financiamento. "São R$ 16 bilhões que estão aguardando a fonte de recursos e nós estamos trabalhando demoradamente e insistentemente na busca de recursos para garantir que as conquistas sejam efetivas", disse.Os profissionais beneficiados pelo projeto são enfermeiros, técnicos em enfermagem e parteiras."""

In [82]:
TEXT2 =  """O ex-Secretário Nacional de Incentivo e Fomento à Cultura, André Porciúncula, revelou na manhã desta terça-feira (03) que a produtora Califórnia, da esposa de Daniela Mercury, recebeu incríveis R$ 1,8 milhões da Lei Rouanet em governos anteriores.A produtora Califórnia, da esposa da Daniela Mercury, recebeu 1,8 milhões da Rouanet dos governos antigos. Mas é claro que a Daniela está fazendo campanha para o ex-presidiário por puto amor ao Lula — escreveu Porciuncula, que também é ex-capitão da Polícia Militar da Bahia.Em outra postagem, o ex-secretário ainda revelou que o Instituto Sol da Liberdade, que pertence a cantora baiana, também recebeu 2,8 milhões da Lei Rouanet.Não é só a produtora da esposa da Daniela Mercury que lucrava milhões com a Lei Rouanet, o instituto que ela comanda costumava pegar milhões, antes do Governo Bolsonaro e as mudanças que fizemos na Rouanet. Sempre presuma uma teta vazia nos críticos do presidente — Porciuncula."""

In [83]:
predict(TEXT)

(1, 47035)


/home/fabricio/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/fabricio/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1

In [58]:
# salvando modelo
with open('classifier_news.pkl', 'wb') as files:
    pickle.dump(model, files)

In [2]:
# load modelo
with open('classifier_news.pkl' , 'rb') as f:
    lr = pickle.load(f)

In [84]:
# salvando tfdf
#with open('classifier_news.pkl', 'wb') as files:
    #pickle.dump(model, files)
    
pickle.dump(cv, open('tfidfvect.pkl', 'wb'))

In [89]:
pickle.dump(model, open('model.pkl', 'wb'))

In [85]:
joblib_vect = pickle.load(open('tfidfvect.pkl', 'rb'))

In [92]:
T = ["adélio","bispo","autor","facada","presidente","jair","bolsonaro","pl","2018","deve","passar","nova","perícia","médica","próximo","mês","avaliar","ainda","representa","perigo","sociedade","pode","ser","solto","solicitação","nova","análise","feita","mpf","ministério","público","federal","nesta","semanaquando","considerado","inimputável","2018","adélio","absolvido","acusações","porém","prevenção","internado","tempo","indeterminado","sentença","fixado","prazo","mínimo","três","anos","submeta","nova","perícia","considerando","decisão","14","abril","2019","partir","14","junho","deste","ano","adélio","pode","passar","nova","análisepor","meio","nota","dpu","defensoria","pública","união","ressaltou","caso","nova","perícia","conclua","fim","risco","sociedade","ainda","caberá","recurso","eventual","desinternação","senhor","adélio","bispo","somente","possível","partir","trânsito","julgado","decisão","determinou","desinternação","pondera","dpu","notaé","importante","mencionar","desinternação","sempre","condicional","devendo","ser","restabelecida","internação","agente","antes","decurso","ano","praticar","fato","indicativo","persistência","periculosidade","conforme","previsto","art","97","código","penal","acrescentou","defensorialaudo","psicológico","comprovou","autor","atentado","contra","hoje","presidente","república","transtorno","delirante","persistente","inquéritos","polícia","federal","demonstraram","agiu","sozinho","tentativa","matar","bolsonaroem","carta","enviada","dpu","revelada","uol","fevereiro","último","adélio","bispo","pede","supremo","tribunal","federal","stf","avalie","limites","punição","cumpre","penitenciária","federal","campo","grande","ms","adélio","defesa","deveria","pedir","ministros","averiguem","internação","data","acabar","inconstitucionalé","diz","duas","cartas","escreveu","serem","entregues","defensoria","pública","união","segundo","uol","apurou","redigidas","dentro","cadeia","circularam","mãos","funcionários","presídio","responsável","encaminhála","defensor","adélionuma","cartas","escritas","mão","adélio","argumenta","pode","ficar","isolado","tempo","prevê","lei","hoje","detenção","máxima","40","anosem","outra","carta","escreveu","defensoria","adélio","reclama","quantidade","medicamentos","toma","diz","mudar","tratamento","médico","43","anos","atendido","médicos","psiquiatras","carta","pede","atendido","psicólogos"]

In [93]:
val_pkl = joblib_vect.transform(T).toarray()

In [94]:
val_pkl

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [95]:
lr.predict(val_pkl)[0]

/home/fabricio/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


0

In [30]:
def process_text(text):
    not_punc = [char for char in text if char not in  string.punctuation]
    not_punc = "".join(not_punc)
    
    clean_words = [word for word in not_punc.split() if word.lower() not in stopwords.words('portuguese') ]
    return clean_words

In [37]:
#process_text(TEXT)

In [115]:
classifier = MultinomialNB()
classifier.fit(Xtrain, ytrain)

MultinomialNB()

In [116]:
print(classifier.predict(Xtrain))
print(Xtrain.values)

[1 0 0 ... 1 1 1]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [45]:
from sklearn.metrics import classification_report

In [118]:
predic = classifier.predict(Xtrain)
print(classification_report(ytrain, predic))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94      2525
           1       0.92      0.97      0.94      2515

    accuracy                           0.94      5040
   macro avg       0.94      0.94      0.94      5040
weighted avg       0.94      0.94      0.94      5040



In [119]:
predic = classifier.predict(Xtest)
print(classification_report(ytest, predic))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      1075
           1       0.83      0.89      0.86      1085

    accuracy                           0.86      2160
   macro avg       0.86      0.86      0.86      2160
weighted avg       0.86      0.86      0.86      2160



In [46]:
preds = model.predict(Xtrain)
print(classification_report(ytrain, preds))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      2527
           1       0.96      0.97      0.96      2513

    accuracy                           0.96      5040
   macro avg       0.96      0.96      0.96      5040
weighted avg       0.96      0.96      0.96      5040



In [47]:
preds = model.predict(Xtest)
print(classification_report(ytest, preds))

              precision    recall  f1-score   support

           0       0.88      0.91      0.90      1073
           1       0.91      0.88      0.89      1087

    accuracy                           0.89      2160
   macro avg       0.90      0.89      0.89      2160
weighted avg       0.90      0.89      0.89      2160



In [122]:
def predictNB(new_text):
    
    """
    Takes the pre-trained model pipeline and predicts new artist based on unseen text.
    
    Parameters
    ----------
    model : Trained scikit-learn model pipeline.
    new_text : str
    
    Returns
    ---------
    prediction : str
    
    """
    article = [new_text]
    # transform song into vector matrix
    new_article_vecs = cv.transform(article)
    ynew = new_article_vecs.todense()
    
    prediction = classifier.predict(ynew)
    
    return prediction[0]

In [124]:
predictNB(TEXT)

/home/fabricio/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
/home/fabricio/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1

### Cross Validation

In [134]:
model_LR = LogisticRegression(max_iter=300)

In [135]:
acuracias = cross_val_score(model_LR, X, y, cv=5, scoring="accuracy")
print("acuracias:", acuracias)
print("acuracia final:", np.mean(acuracias), "+-", np.std(acuracias))

acuracias: [0.91527778 0.88958333 0.92083333 0.91666667 0.90138889]
acuracia final: 0.9087499999999998 +- 0.011600340565456162


In [136]:
model_LR.fit(Xtrain, ytrain)

LogisticRegression(max_iter=300)

array([0.91527778, 0.88958333, 0.92083333, 0.91666667, 0.90138889])

In [130]:
acuracias.mean()

0.9087499999999998

In [131]:
acuracias_classifier = cross_val_score(classifier, X, y, cv=5, scoring="accuracy")

In [132]:
acuracias_classifier

array([0.87847222, 0.85763889, 0.86805556, 0.87569444, 0.85416667])

In [133]:
acuracias_classifier.mean()

0.8668055555555556

In [11]:
cv = TfidfVectorizer(
            stop_words=stopwords.words('portuguese'), analyzer='word',
            ngram_range=(1, 1), lowercase=True, use_idf=True)

In [12]:
def process_text(text):
    
    not_punc = [char for char in text if char not in  string.punctuation]
    not_punc = "".join(not_punc)
    
    clean_words = [word.lower() for word in not_punc.split() if word.lower() not in stopwords.words('portuguese') ]
    # cv.fit(clean_words)
    # print('aqui foi ', a)
    return clean_words


In [13]:
def predictLR(new_text):
    
    """
    Takes the pre-trained model pipeline and predicts new artist based on unseen text.
    
    Parameters
    ----------
    model : Trained scikit-learn model pipeline.
    new_text : str
    
    Returns
    ---------
    prediction : str
    
    """
    text = process_text(new_text)
    new_text = [new_text]
    print(new_text)
    # transform song into vector matrix
    new_article_vecs = cv.transform(text)
    ynew = new_article_vecs.todense()
    print(ynew)
    prediction = lr.predict(ynew)
    
    return prediction[0]

In [16]:
texto_uol = """O presidente do Senado, Rodrigo Pacheco (PSD-MG), derrubou uma tentativa de governistas de chamar o ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF), para falar sobre os inquéritos que investigam atos antidemocráticos e o uso de fake news no plenário da Casa.O senador Eduardo Girão (Pode-CE) apresentou um requerimento para a realização de uma sessão de debates temáticos com Moraes, na tentativa de pressionar o ministro. O pedido recebeu apoio de governistas, aumentando as investidas do grupo contra Moraes diante dos ataques do presidente Jair Bolsonaro ao Judiciário. Pacheco impugnou o requerimento, impedindo a realização da audiência com o ministro.De acordo com o presidente do Senado, o pedido dos governistas contraria a Constituição e os propósitos previstos no regimento do Senado para a realização de debates temáticos, abrindo caminho para um "controle indireto de atos jurisdicionais" por meio de uma sessão com o magistrado. Girão recorreu da decisão de Pacheco. O presidente da Casa não julgou o recurso."""

In [26]:
tf = TfidfVectorizer(
            stop_words=stopwords.words('portuguese'), analyzer='word',
            ngram_range=(1, 1), lowercase=True,binary=True, use_idf=True)



#texto_uol

In [27]:
tf.fit(texto_uol.split(','))


TfidfVectorizer(binary=True,
                stop_words=['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em',
                            'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se',
                            'na', 'por', 'mais', 'as', 'dos', 'como', 'mas',
                            'ao', 'ele', 'das', 'à', 'seu', 'sua', ...])

In [53]:
predict(texto_uol)

/home/fabricio/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/fabricio/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


1

In [31]:
corpus_vecs = tf.transform(texto_uol.split(','))

ynew = corpus_vecs.todense()
#lr.predict()
ynew.shape

(10, 72)

In [15]:
predictLR(texto_uol)

['O presidente do Senado, Rodrigo Pacheco (PSD-MG), derrubou uma tentativa de governistas de chamar o ministro Alexandre de Moraes, do Supremo Tribunal Federal (STF), para falar sobre os inquéritos que investigam atos antidemocráticos e o uso de fake news no plenário da Casa.O senador Eduardo Girão (Pode-CE) apresentou um requerimento para a realização de uma sessão de debates temáticos com Moraes, na tentativa de pressionar o ministro. O pedido recebeu apoio de governistas, aumentando as investidas do grupo contra Moraes diante dos ataques do presidente Jair Bolsonaro ao Judiciário. Pacheco impugnou o requerimento, impedindo a realização da audiência com o ministro.De acordo com o presidente do Senado, o pedido dos governistas contraria a Constituição e os propósitos previstos no regimento do Senado para a realização de debates temáticos, abrindo caminho para um "controle indireto de atos jurisdicionais" por meio de uma sessão com o magistrado. Girão recorreu da decisão de Pacheco. O 

NotFittedError: The TF-IDF vectorizer is not fitted

######### 

In [2]:
with open('classifier_news.pkl' , 'rb') as f:
    classifier_news = pickle.load(f)

In [3]:
def process_text(text):
    not_punc = [char for char in text if char not in  string.punctuation]
    not_punc = "".join(not_punc)
    
    clean_words = [word for word in not_punc.split() if word.lower() not in stopwords.words('portuguese') ]
    return clean_words


def vectorized_text(text):
    cv = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, use_idf=True, norm='l2', smooth_idf=True)
    cv.fit(text)
    corpus_vecs = cv.fit_transform(text)
    ynew = corpus_vecs.todense()
    return ynew

In [4]:
TXT = """O ex-governador de São Paulo Geraldo Alckmin (PSB) testou positivo para covid-19, um dia antes do lançamento da chapa do ex-presidente Luiz Inácio Lula da Silva (PT) ao Palácio do Planalto. O UOL confirmou a informação com a pré-campanha de Lula.Segundo o deputado José Guimarães (PT-CE), Alckmin avisou à cúpula petista do diagnóstico, mas o comitê de campanha decidiu manter o ato político no sábado (7), em São Paulo, e o ex-governador deve participar de forma virtual.Por meio de nota, a assessoria de comunicação do ex-governador de São Paulo, que foi vacinado contra a covid-19 com as três doses, disse que ele está em casa e passa bem. A assessoria também confirmou que o evento está mantido e ele pode participar de forma remota.A previsão é de que o evento de pré-candidatura reúna 4.000 convidados, entre políticos em geral, pré-candidatos nos estados, artistas, empresários, sindicalistas e representantes da sociedade civil. O lançamento acontecerá no Expo Center Norte, às 10h deste sábado.Sete partidos se aliarão oficialmente: PT, PSB, PC do B, PSOL, Rede, PV e Solidariedade. Representantes de outros partidos, como PSD, Avante e MDB, também foram convidados para o ato de sábado.Alckmin esteve com Lula e vários outros petistas na segunda-feira (2) em reuniões de avaliação. Dois dias depois, ele se reuniu com o ex-ministro Aloizio Mercadante para discutir o programa de governo de sua chapa com Lula. O encontro ocorreu na Fundação Perseu Abramo, em São Paulo.Na reunião, Mercadante pediu indicações de nomes técnicos para debater o programa de governo e sugestões ao ex-governador, que defendeu a necessidade de se construir um "pacto republicano" no Brasil. Alckmin falou também do agronegócio. Parte do setor rejeita Lula e prefere o presidente Jair Bolsonaro (PL), mas o ex-governador tem tentado reduzir essa resistência ao petista no setor rural.Na próxima semana, estava prevista uma viagem de Alckmin a Minas Gerais, com Lula, mas o ex-governador também não poderá participar.*Com Estadão Conteúdo e Reuters"""

In [10]:
clean_text = process_text(TEXT)
vector_text = vectorized_text(clean_text)

In [8]:
classifier_news

LogisticRegression(max_iter=300)

In [11]:
prediction = classifier_news.predict(vector_text)

/home/fabricio/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/fabricio/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


ValueError: X has 206 features, but LogisticRegression is expecting 47035 features as input.

In [86]:
def predictNB(new_text):
    
    """
    Takes the pre-trained model pipeline and predicts new artist based on unseen text.
    
    Parameters
    ----------
    model : Trained scikit-learn model pipeline.
    new_text : str
    
    Returns
    ---------
    prediction : str
    
    """
    new_text = process_text(new_text)
    print(len(new_text))
    cv = TfidfVectorizer()
    cv.fit(new_text)
    # transform song into vector matrix

    new_article_vecs = cv.transform(new_text)
    ynew = new_article_vecs.todense()
    
    f = np.transpose(ynew)
    print(f)
    
    prediction = model.predict(ynew)
        
    return prediction[0]

In [87]:
predictNB(TXT)

190
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/home/fabricio/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/fabricio/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


ValueError: X has 153 features, but LogisticRegression is expecting 47035 features as input.